In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
!ls csv_files

yelp_academic_dataset_business.csv  yelp_academic_dataset_tip_transposed.csv
yelp_academic_dataset_review.csv    yelp_academic_dataset_user.csv
yelp_academic_dataset_tip.csv


In [3]:
generic = lambda x: ast.literal_eval(x)
conv = {'friends': generic}

In [4]:
business = pd.read_csv('csv_files/yelp_academic_dataset_business.csv')
review = pd.read_csv('csv_files/yelp_academic_dataset_review.csv')
tip = pd.read_csv('csv_files/yelp_academic_dataset_tip_transposed.csv')
user = pd.read_csv('csv_files/yelp_academic_dataset_user.csv',converters=conv)

In [5]:
business.index += 1
review.index += 1
tip.index += 1
user.index += 1

# Parse user

In [6]:
user_ids = user['user_id'].reset_index().set_index('user_id')['index'].to_dict()

In [7]:
user_table = user

In [8]:
friends = user['friends'].map(lambda friends: list(map(lambda x: user_ids[x],friends)))

In [9]:
elite = user[user['elite'].map(lambda x:type(x))==type("")]['elite'].map(lambda e:list(map(lambda x:int(x),e.split(","))))

In [10]:
user_table.head()

1    [292, 126239, 272957, 30954, 18898, 1392, 2805...
2    [293791, 578623, 205429, 607173, 586445, 58460...
3    [47613, 111730, 458447, 54553, 6554, 19364, 12...
4    [589227, 495881, 434834, 769828, 117698, 19492...
5    [26968, 431107, 354309, 48664, 1557, 215430, 3...
Name: friends, dtype: object

# Parse business

In [11]:
business_ids = business['business_id'].reset_index().set_index('business_id')['index'].to_dict()

# Parse review

In [12]:
review_ids = review['review_id'].reset_index().set_index('review_id')['index'].to_dict()

In [13]:
excluded_review = review[(review.user_id.isin(user_ids.keys()) & review.business_id.isin(business_ids.keys())).map(lambda x:not x)]

In [14]:
included_review = review[review.user_id.isin(user_ids.keys()) & review.business_id.isin(business_ids.keys())]

In [15]:
excluded_review

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
199242,41b2SLmjLcxTGLVRxASiDA,0,2006-04-11 09:05:18,0.0,WWYQ1ce6mNt7AvRHu8w-jQ,3.0,Rating purely on food and 18th hole view: 5 stars,NaN,NaN
199243,NaN,1,53bZ_EsXH71L7iFs5MP9_w,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
included_review.loc[:,'user_id'] = included_review.apply(lambda x: user_ids[x.user_id], axis=1)

/home/bastien/.local/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
included_review.loc[:,'business_id'] = included_review.apply(lambda x: business_ids[x.business_id], axis=1)

In [18]:
review_table = included_review

In [19]:
review_table.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
1,107045,0,2016-11-09 20:09:03,0.0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3.0,65413
2,30138,0,2018-01-30 23:07:38,0.0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7.0,8027
3,112123,5,2016-05-07 01:21:02,4.0,G7XHMxG0bx9oBJNECG4IFg,3.0,Tracy dessert had a big name in Hong Kong and ...,5.0,11467
4,87448,0,2017-12-15 23:27:08,1.0,-I5umRTkhw15RqpKMl_o1Q,1.0,"Walked in around 4 on a Friday afternoon, we s...",0.0,29616
5,15683,0,2014-12-17 19:04:33,0.0,0AsmPiAQduxh5jE_si8cLA,5.0,ended up here because Raku was closed and it r...,0.0,21932


# Parse tip

In [20]:
excluded_tip = tip[(tip.user_id.isin(user_ids.keys()) & tip.business_id.isin(business_ids.keys())).map(lambda x:not x)]

In [21]:
included_tip = tip[tip.user_id.isin(user_ids.keys()) & tip.business_id.isin(business_ids.keys())]

In [22]:
excluded_tip

,business_id,compliment_count,date,text,user_id


In [23]:
included_tip.loc[:,'user_id'] = included_tip.apply(lambda x: user_ids[x.user_id], axis=1)

In [24]:
included_tip.loc[:,'business_id'] = included_tip.apply(lambda x: business_ids[x.business_id], axis=1)

In [25]:
tip_table = included_tip

In [26]:
tip_table.head()

,business_id,compliment_count,date,text,user_id
1,19903,0,2014-03-27 03:51:24,"Great for watching games, ufc, and whatever el...",2221
2,121777,0,2013-05-25 06:00:56,Happy Hour 2-4 daily with 1/2 price drinks and...,38143
3,152691,0,2011-12-26 01:46:17,Good chips and salsa. Loud at times. Good serv...,27342
4,10040,0,2014-03-23 21:32:49,The setting and decoration here is amazing. Co...,3217
5,109424,0,2012-10-06 00:19:27,Molly is definately taking a picture with Sant...,30346
